In [6]:
#Q1. Running Elastic
import requests 
import tiktoken
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json() 

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  

In [8]:
from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
#Q1. Running Elastic
es_client.info()

ObjectApiResponse({'name': '33ffe1029e06', 'cluster_name': 'docker-cluster', 'cluster_uuid': '2BB4LMhZTR-mkmpHkPASTw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [11]:
#Q2. Indexing the data
index_settings200 = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name200 = 'course_questions200'

es_client.indices.create(index=index_name200, body=index_settings200)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions200'})

In [16]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name200,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [19]:

def elastic_searchq3(query):
    search_query = {
        "size": 5, 
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query , 
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields",
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name200, body=search_query)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [21]:
#Q3. Searching
query = "How do execute a command on a Kubernetes pod?"
search_resultsq3 = elastic_searchq3(query)
print(search_resultsq3[0])

44.50556


In [38]:
def elastic_searchq4(query):
    search_query = {
        "size": 3,  # Return exactly 3 results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query ,  # Add related terms for better matching
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields",
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name200, body=search_query)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [39]:
query = "How do copy a file to a Docker container?"
search_resultsQ4 = elastic_searchq4(query)
print(search_resultsQ4)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'm

In [40]:
#Q4. Filtering
print("\nValue from the key 'question':")
for item in search_resultsQ4:
    print(item['question'])


Value from the key 'question':
How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


In [33]:
index_settingsq400 = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_nameq400 = 'course_questionsq400'

es_client.indices.create(index=index_nameq400, body=index_settingsq400)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questionsq400'})

In [34]:
from tqdm.auto import tqdm

In [35]:
for doc in tqdm(search_resultsQ4):
    es_client.index(index=index_nameq400,document=doc)

  0%|          | 0/3 [00:00<?, ?it/s]

In [41]:
def elastic_search(query):
    search_queryn = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_nameq400, body=search_queryn)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [42]:
#Q5. Building a prompt
def build_prompt(query, search_results):
    # Define the templates
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

 
    context = ""
    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    
 
    context = context.strip()
    

    prompt = prompt_template.format(question=query, context=context)
    
    return prompt

In [43]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [46]:
#Q6. Tokens
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    print(f"The len of the promt is: {len(prompt)}")
    encoding = tiktoken.encoding_for_model("gpt-4o")
    tokens = encoding.encode(prompt)
    token_count = len(tokens)
    print(f"The number of tokens are: {token_count}")
    answer = llm(prompt)
    return answer

In [47]:
query = "How to copy a file to a Docker container?"
result = rag(query)

The len of the promt is: 1446
The number of tokens are: 320


In [48]:
print(result)

To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax to copy a file or directory from your local machine into a running Docker container is as follows:

```bash
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```
